# Introduction

Considerable amount of microbial community samples are from understudied biomes which were more context-dependent and represent the development of gut microbiome. To prove the capability of EXPERT for distinguishing more detailed biomes, we used EXPERT to source track the succession of infant gut microbiome.

The dataset was collected by Backhed et al.(accession number [ERP005989](https://www.ebi.ac.uk/ena/browser/view/PRJEB6456)), which contains fecal samples of 98 Swedish infants at birth, 4 months and 12 months after birth, and corresponding mothers during the first few days after delivery. Among all the infants, 83 were vaginally delivered, and 15 were born by cesarean section. Data are processed using metagenic shotgun sequencing.

By cross-validation, we assessed the performance of different models. We performed proportional sampling cross-validation by randomly select 10% of the samples from mother, infants at birth, 4 monthes and 12 months of age as query, respectively. This process was repeated for five times. The model built from general model showed the best performance when source tracking the samples.

# Reproducibility statement

- EXPERT supports completely reproducible optimization & inference.
- Processed data are provided for reproducing the result, the original data can be found under `dataFiles/`.
- Rerunning the entire notebook with the configuration below should yield **completely consistent** results (compared to those reported in our paper).
- Session information
    - EXPERT (version 0.3)
    - Python (version 3.8.2)
    - TensorFlow (version 2.3.1)
    - Pandas (version 1.1.3)
    - NumPy (version 1.18.5)
    - ETE3 (version 3.1.2)
    - NCBI taxonomy database (released [2020-09-01](https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump_archive/))

# Process
The following sections are used to reproduce the cross-validation result of infant succession part, which reported in our paper. For detailed configuration and interpretation of results, please read our original paper first.

All the files are in Infant-succession-/experiments/exp_5_source_tracking.

### Optimization
- `--finetune`: enable finetune for further optimization.
- `--update-statistics`: update statistics for Z-score standardization.

In [ ]:
%%bash
for i in {0,1,2,3,4}; do
    expert train -i experiments/exp_$i/SourceCM.h5 -t ontology.pkl \
    -l experiments/exp_$i/SourceLabels.h5 -o experiments/exp_$i/Independent;

done;

In [ ]:
%%bash
for i in {0,1,2,3,4}; do 
expert transfer -i experiments/exp_$i/SourceCM.h5 -t ontology.pkl \
        -l experiments/exp_$i/SourceLabels.h5 -o experiments/exp_$i/Transfer_HM \
        -m ../Human-assessment/experiments/exp_1/Independent
        --finetune --update-statistics;
done;

In [ ]:
%%bash
for i in {0,1,2,3,4}; do    
        expert transfer -i experiments/exp_$i/SourceCM.h5 -t ontology.pkl \
        -l experiments/exp_$i/SourceLabels.h5 -o experiments/exp_$i/Transfer_GM \
        --finetune --update-statistics;
done;

### Quantifying source contributions

- `--measure-unknown`: measure the contribution from unknown source(s).

In [ ]:
%%bash
for i in {0,1,2,3,4}; do
    expert search -i experiments/exp_$i/QueryCM.h5 -m experiments/exp_$i/Independent \
    -o experiments/exp_$i/Search_Independent \
    --measure-unknown;
done;

In [ ]:
%%bash
for i in {0,1,2,3,4}; do
    expert search -i experiments/exp_$i/QueryCM.h5 -m experiments/exp_$i/Transfer_HM \
    -o experiments/exp_$i/Search_Transfer_HM \
    --measure-unknown;
done;

In [ ]:
%%bash
for i in {0,1,2,3,4}; do
    expert search -i experiments/exp_$i/QueryCM.h5 -m experiments/exp_$i/Transfer_GM \
    -o experiments/exp_$i/Search_Transfer_GM \
    --measure-unknown;
done;

### Evaluating performances
- `-S`: Set threshold for evaluation

In [ ]:
%%bash
for i in {0,1,2,3,4}; do
    expert evaluate -i experiments/exp_$i/Search_Independent -l experiments/exp_$i/QueryLabels.h5 \
    -o experiments/exp_$i/Eval_Independent -S 0;
    
done;

In [ ]:
%%bash
for i in {0,1,2,3,4}; do
    expert evaluate -i experiments/exp_$i/Search_Transfer_HM -l experiments/exp_$i/QueryLabels.h5 \
    -o experiments/exp_$i/Eval_Transfer_HM -S 0;

done;

In [ ]:
%%bash
for i in {0,1,2,3,4}; do
    expert evaluate -i experiments/exp_$i/Search_Transfer_GM -l experiments/exp_$i/QueryLabels.h5 \
    -o experiments/exp_$i/Eval_Transfer_GM -S 0;

done;

## Support
For support reproducing the result, please email: huichong.me@gmail.com.